In [1]:
from dendron import *
from dendron.actions.causal_lm_action import CausalLMActionConfig, CausalLMAction

In [3]:
cfg = CausalLMActionConfig(load_in_4bit=True, 
                           max_new_tokens=64, 
                           do_sample=True, 
                           top_p=0.95, 
                           use_flash_attn_2=True,
                           model_name = 'v1olet/v1olet_merged_dpo_7B')
                           #model_name="mistralai/Mistral-7B-Instruct-v0.2")
node = CausalLMAction("lm_action", cfg)
tree = BehaviorTree("causal-lm-tree", node)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#prompt = "<s>[INST]{query}[/INST]" For Mistral

prompt = """{system}
### Instruction:
{query}

### Response:
"""

import re
def trunc(node, s):
    # for mistral:
    #m = re.search(r'\[INST\](.*)\[/INST\](.*)', s, re.DOTALL)
    #return m.group(2)

    m = re.search(r'### Response:(.*)', s, re.DOTALL)
    return m.group(1).strip()

tree.root.set_output_processor(trunc)

In [5]:
import re

def trunc(node):
    out_str = node.blackboard[node.output_key]
    m = re.search(r'### Response:(.*)', out_str, re.DOTALL)
    node.blackboard[node.output_key] = m.group(1).strip()

tree.root.set_post_tick(trunc)

In [6]:
tree.blackboard_set("in", prompt.format(system="You are busy and don't have much time to be verbose.", 
                                        query="The following is a headline from a newspaper: 'History-making SpaceX booster mostly destroyed in post-flight topple.' Is this about a spacecraft? Give a one-word yes or no."))

In [7]:
status = tree.tick_once()
if status == NodeStatus.SUCCESS:
     print(tree.blackboard["out"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/richard/.conda/envs/test_dendron/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Yes.


In [8]:
print(tree.blackboard)

Key              | Description      | Type             | Value            |
in               | Autogenerated en | <class 'str'>    | You are busy and | 
out              | Autogenerated en | <class 'str'>    | Yes.             | 



In [9]:
tree.blackboard['out']

'Yes.'